# CH09 Common design patterns

#### 실전 속의 디자인 패턴

##### 생성(creational) 페턴
:객체 초기화를 위한 파라미터를 결정하거나  
 초기화에 필요한 관련 객체를 준비하는 등의 모든 관련 작업을 단순화하려는 것

##### 팩토리
: 파이썬은 클래스, 함수, 사용자 정의 객체 등의 역할이 구분되지 않으므로 별로 필요하지 않음

##### 싱글턴과 공유 상태(monostate)
- 일반적으로 싱글턴은 사요하지 않는 것이 좋음
- 객체 지향 소프트웨어를 위한 전역 변수의 한 형태이며 나쁜 습관(대부분)
- 파이썬에서 이를 해결하는 가장 쉬운 방법은 모듈을 사용하는 것  
 여러 번 임포트하더라도 sys.modules에 로딩되는 것은 항상 하나임
- 클래스 변수의 게터나 세트를 활용하거나 디스크립터 사용할 수도 있음


In [5]:
import logging

import logging

logging.basicConfig()
logger = logging.getLogger(__name__)

class GitFetcher:
    _current_tag = None

    def __init__(self, tag):
        self.current_tag = tag # 프로퍼티를 통해 클래스 변수와 연결되어 있음

    @property
    def current_tag(self):
        if self._current_tag is None:
            raise AttributeError("Tag is not initialized yet.")
        return self._current_tag 
    
    @current_tag.setter
    def current_tag(self, new_tag):
        self.__class__._current_tag = new_tag 
        
    def pull(self):
        logger.info(f"Pull at {self.current_tag}")

In [3]:
f1 = GitFetcher(0.1)
f2 = GitFetcher(0.2)
f1.current_tag = 0.3
f2.pull()

In [4]:
f1.pull()

더 많은 속성이 필요하거나 더 깔끔한 디자인 -> 디스크립터 사용

In [21]:
import logging

logging.basicConfig()
logger = logging.getLogger(__name__)

class SharedAttribute:
    def __init__(self, initial_value=None):
        self.value = initial_value
        self._name = None

    def __get__(self, instance, owner):
        if instance is None:
            return self
        if self.value is None:
            raise AttributeError(f"{self._name} was never set")
        return self.value

    def __set__(self, instance, new_value):
        self.value = new_value

    def __set_name__(self, owner, name):
        self._name = name
 

class GitFetcher:

    current_tag = SharedAttribute()
    current_branch = SharedAttribute()

    def __init__(self, tag, branch=None):
        self.current_tag = tag
        self.current_branch = branch

    def pull(self):
        logger.info("pulling from %s", self.current_tag)
        return self.current_tag

##### # borg 패턴
- 어쩔 수 없이 꼭 싱글턴을 사용해야 하는 최후의 더 나은 대안.  
- 같은 클래스의 모든 인스턴스가 모든 속성을 복제하는 객체를 만드는 것.

In [7]:
class BaseFetcher:
    def __init__(self, source):
        self.source = source

        
class TagFetcher(BaseFetcher):
    _attributes = {} # borg 패턴의 특징,
    
    def __init__(self, source):
        self.__dict__ = self.__class__._attributes
        super().__init__(source)
        
    def pull(self):
        logger.info(f"Pull at tag {self.source}")
        return f"Tag = {self.source}"
    

class BranchFetcher(BaseFetcher):
    _attributes = {} # 사전은 레퍼런스 형태로 전달되는 변경 가능한 mutable 객체이므로
                     # 사전을 업데이트하면 모든 객체에 동일하게 업데이트 됨
    
    def __init__(self, source):
        self.__dict__ = self.__class__._attributes
        super().__init__(source)
        
    def pull(self):
        logger.info(f"Pull at branch {self.source}")
        return f"Branch = {self.source}"    

속성을 저장할 사전을 클래스 속성으로 지정하고, 객체를 초기화할 때 모든 객체에서 바로 이 동일한 사전을 참조하도록 해야 한다.

In [8]:
class SharedAllMixin:
    def __init__(self, *args, **kwargs):
        try:
            self.__class__._attributes
        except AttributeError:
            self.__class__._attributes = {}

        self.__dict__ = self.__class__._attributes
        super().__init__(*args, **kwargs)


class BaseFetcher:
    def __init__(self, source):
        self.source = source



class TagFetcher(SharedAllMixin, BaseFetcher):
    def pull(self):
        logger.info("pulling from tag %s", self.source)
        return f"Tag = {self.source}"


class BranchFetcher(SharedAllMixin, BaseFetcher):
    def pull(self):
        logger.info("pulling from branch %s", self.source)
        return f"Branch = {self.source}"

    
if __name__ == "__main__":
    f1 = TagFetcher(0.1)
    f2 = TagFetcher(0.2)
    
    print(f2.__dict__)
    print(TagFetcher.mro())
    print(SharedAllMixin.__subclasses__())
    
    f2.pull()
    f1.pull()

{'source': 0.2}
[<class '__main__.TagFetcher'>, <class '__main__.SharedAllMixin'>, <class '__main__.BaseFetcher'>, <class 'object'>]
[<class '__main__.TagFetcher'>, <class '__main__.BranchFetcher'>]


##### 빌더
- 필요로 하는 모든 객체를 직접 생성해주는 하나의 복잡한 객체를 만들어야 함
- 한 번에 모든 것을 처리해주는 추상화를 해야 한다는 것

##### 구조페턴
- 여러 개의 객체를 조합하여  만듬 -> 향상된 기능을 깔끔하게 구현

##### 어댑터 패턴
구현 방법
- 사용하려는 클래스를 상속 받는 클래스를 만드는 것  
    개념적으로 상속은 is a 관계로 적용하는 것이 바람직  
    (cuz : 얼마나 많은 외부라이브러리를 가져올지 정확히 알기 어려우므로)

In [ ]:
class UsernameLookup:
    def search(self, user_namespace):
        logger.info("looking for %s", user_namespace)

In [ ]:
class UserSource(UsernameLookup):
    def fetch(self, user_id, username):
        user_namespace = self._adapt_arguments(user_id, username)
        return self.search(user_namespace)

    @staticmethod
    def _adapt_arguments(user_id, username):
        return f"{user_id}:{username}"

- 컴포지션 사용 (더 나은 방법)

In [ ]:
class UserSource:
    def __init__(self, username_lookup: UsernameLookup) -> None:
        self.username_lookup = username_lookup

    def fetch(self, user_id, username):
        user_namespace = self._adapt_arguments(user_id, username)
        return self.username_lookup.search(user_namespace)

    @staticmethod
    def _adapt_arguments(user_id, username):
        return f"{user_id}:{username}"

#### 컴포지트
- 객체는 구조화된 트리 형태, 기본 객체 : 리프노드 | 컨테이너 객체 : 중간 노드
- 클라이언트는 이중에 아무거나 호출하여 결과를 얻고자 함
- 컴포지트 객체도 클라이언트처럼 동작
- 리프 노드인지 중간 노드인지 상관없이 해당 요청을 관련 노드가 처리할 수 있을 때까지 계속 전달

In [1]:
from typing import Iterable, Union


class Product:
    def __init__(self, name, price):
        self._name = name
        self._price = price
        
    @property
    def price(self):
        return self._price
    
    
class ProductBundle:
    def __init__(
        self,
        name,
        perc_discount,
        *products: Iterable[Union[Product, "ProductBundle"]]
    ) -> None:
        self._name = name
        self._perc_discount = perc_discount
        self._products = products

    @property
    def price(self):
        total = sum(p.price for p in self._products)
        return total * (1 - self._perc_discount)

#### 데코레이터(패턴)
- 5장 데코레이터랑은 유사점 있지만 다른 개념
- 상속을 하지 않고도 객체의 기능을 동적으로 확장가능


In [2]:
class DictQuery:
    def __init__(self, **kwargs):
        self._raw_query = kwargs
        
    def render(self)->dict:
        return self._raw_query
    

In [3]:
class QueryEnhancer:
    def __init__(self, query: DictQuery):
        self.decorated = query

    def render(self):
        return self.decorated.render()


class RemoveEmpty(QueryEnhancer):
    def render(self):
        original = super().render()                     # 오리지널을 받아서
        return {k: v for k, v in original.items() if v} # 새로운 형식으로 리턴함


class CaseInsensitive(QueryEnhancer):
    def render(self):
        original = super().render()
        return {k: v.lower() for k, v in original.items()}

In [4]:
original = DictQuery(foo="bar", empty="", none=None, upper="UPPERCASE", title="Title")
new_query = CaseInsensitive(RemoveEmpty(original))

In [5]:
original.render()

{'foo': 'bar',
 'empty': '',
 'none': None,
 'upper': 'UPPERCASE',
 'title': 'Title'}

In [6]:
new_query.render()

{'foo': 'bar', 'upper': 'uppercase', 'title': 'title'}

##### # 파이썬의 동적인 특성을 활요한 다른 방법

In [9]:
from typing import Callable, Dict, Iterable

class QueryEnhancer:
    def __init__(
        self,
        query: DictQuery,
        *decorators: Iterable[Callable[[Dict[str, str]], Dict[str, str]]]
    )-> None:
        self._decorated = query
        self._decorators = decorators
        
    def render(self):
        current_result = self._decorated.render()
        for deco in self._decorators:
            current_result = deco(current_result)
        return current_result
    
def remove_empty(original: dict) -> dict:
    return {k: v for k, v in original.items() if v}

def case_insensitive(original: dict)-> dict:
    return {k: v.lower() for k, v in original.items()}

In [10]:
QueryEnhancer(original, remove_empty, case_insensitive).render()

{'foo': 'bar', 'upper': 'uppercase', 'title': 'title'}

#### 파사드(Facade)
- 객체 간 상호 작용을 단순화하려는 많은 상황에서 유용
- 각각의 객체에 대한 모든 연결을 만드는 대신 파사드 역할을 하는 중간 객체를 만듬
- _ _init_ _.py 파일이 모듈의 루트로서 파사드와 같은 역할을 함.

##### 행동(behavioral) 패턴
: 어떤 패턴을 사용하든지 간에 결국에는 중복을 피하거나 행동을 캡슐화하는 추상화를 통해 모델 간 결합력을 낮추는 방법

##### 책임 연쇄 패턴
- 후계자(successor)라는 개념 추가: 현재 이번트가 처리할 수 없는 경우를 대비한 다음 이벤트 객체
- 직접 처리가 가능한 경우 결과를 반환, 처리가 불가능하면 후계자에게 전달하고 이러한 과정 반복

In [11]:
import re

class Event:
    pattern = None

    def __init__(self, next_event=None):
        self.successor = next_event

    def process(self, logline: str):
        if self.can_process(logline):
            return self._process(logline)

        if self.successor is not None:
            return self.successor.process(logline)

    def _process(self, logline: str) -> dict:
        parsed_data = self._parse_data(logline)
        return {
            "type": self.__class__.__name__,
            "id": parsed_data["id"],
            "value": parsed_data["value"],
        }

    @classmethod
    def can_process(cls, logline: str) -> bool:
        return cls.pattern.match(logline) is not None

    @classmethod
    def _parse_data(cls, logline: str) -> dict:
        return cls.pattern.match(logline).groupdict()


class LoginEvent(Event):
    pattern = re.compile(r"(?P<id>\d+):\s+login\s+(?P<value>\S+)")


class LogoutEvent(Event):
    pattern = re.compile(r"(?P<id>\d+):\s+logout\s+(?P<value>\S+)")

#로그인과 로그아웃 이벤트를 둘 다 처리할 수 있는 타입을 추가
class SessionEvent(Event):
    pattern = re.compile(r"(?P<id>\d+):\s+log(in|out)\s+(?P<value>\S+)")

In [12]:
chain = LogoutEvent(LoginEvent())
chain.process("567: login User")
#사전은 LoginEvent가 생성한 것이다. 로그인

{'type': 'LoginEvent', 'id': '567', 'value': 'User'}

In [ ]:
# 애플리케이션에서LoginEvent 전에SessionEvent 를먼저처리하고싶은경우
chain = SessionEvent (LoginEvent (LogoutEvent()))

#### 템플릿 메서드 패턴(template method pattern)
#참고 : p296.  
코드 재사용성을 높여주고 객체를 유현하게 하여 다형성 유지 + 코드 쉽게 수정

주요 개념 : 어떤 행위를 정의할 때 특정한 형태의 클래스 계층구조 만듬

##### 커맨드(command pattern)
: 수행해야 할 작업을 요청한 순간부터 실행 시까지 분리할 수 있는 기능 제공,  
클라이언트가 발행한 원래 요청을 수신자와 분리도 가능

- 실행될 명령의 파라미터를 저장하는 객체를 만드는 것
- 명령에 필요한 파라미터에 필터를 더하거나 제거하는 것처럼 상호작용할 수 있는 메서드 제공
- 마지막으로 실제로 작업을 수행할 객체를 만듬

#### 상태(state) 패턴
: 상태(state) 패턴은 구체화(reification) 을 도와주는 패턴.  
상태별로 작은 객체를 만들어 각각의 객체가 적은 책임을 갖게 하는 것

In [22]:
import abc

import logging

logging.basicConfig()
logger = logging.getLogger(__name__)

class InvalidTransitionError(Exception):
    """Raised when trying to move to a target state from an unreachable sourcer"""
    

class MergeRequestState(abc.ABC):
    def __init__(self, merge_request):
        self._merge_request = merge_request
        
    @abc.abstractclassmethod
    def open(self):
        ...
    
    @abc.abstractclassmethod
    def close(self):
        ...
    
    @abc.abstractclassmethod
    def merge(self):
        ...
        
    def __str__(self):
        return self.__class__.__name__
    
class Open(MergeRequestState):
    def open(self):
        self._merge_request.approvals = 0

    def close(self):
        self._merge_request.approvals = 0
        self._merge_request.state = Closed 

    def merge(self):
        logger.info("merging %s", self._merge_request)
        logger.info("deleting branch %s", self._merge_request.source_branch)
        self._merge_request.state = Merged 
        
        
class Closed(MergeRequestState):
    def open(self):
        logger.info("reopening closed merge request %s", self._merge_request)
        self._merge_request.state = Open # 상태를 전환하는 코드

    def close(self):
        """Current state."""

    def merge(self):
        raise InvalidTransitionError("can't merge a closed request")
    
class Merged(MergeRequestState):
    def open(self):
        raise InvalidTransitionError("already merged request")

    def close(self):
        raise InvalidTransitionError("already merged request")

    def merge(self):
        """Current state."""
        
        

class MergeRequest:
    def __init__(self, source_branch: str, target_branch: str) -> None:
        self.source_branch = source_branch
        self.target_branch = target_branch
        self._state = None
        self.approvals = 0
        self.state = Open

    @property
    def state(self):
        return self._state

    @state.setter
    def state(self, new_state_cls):
        print("calling setter", new_state_cls)
        self._state = new_state_cls(self)

    def open(self):
        return self.state.open()

    def close(self):
        return self.state.close()

    def merge(self):
        return self.state.merge()

    def __str__(self):
        return f"{self.target_branch}:{self.source_branch}"


In [23]:
mr = MergeRequest("develop", "master")
mr.open()
mr.approvals

calling setter <class '__main__.Open'>


0

In [24]:
mr.approvals = 3
mr.close()
mr.approvals

calling setter <class '__main__.Closed'>


0

In [25]:
mr.open()

calling setter <class '__main__.Open'>


In [26]:
mr.merge()

calling setter <class '__main__.Merged'>


In [27]:
mr.close()

InvalidTransitionError: already merged request

#### Null 객체 패턴

- 함수나 메서드는 일관된 탑입을 반환해야 함

Ex) 사용자를 조회했는데 사용자가 없을 때
- 예외 발생
- UserUnknown 타입을 반환

그 어떠한 경우에도 None을 반환하면 안됨  
None이라는 문구는 방금 일어난 일에 대한 아무것도 설명해주지 않으며  
호출자는 특별한 공지가 없으면 아무 생각없이 반환 객체에 대해 메서드를 호출할 것이므로.  
결국 AttributeError가 발생( None -> AttributeError까지 이어짐)